# INITIAL CONF

In [101]:
#maxizeallresources
import findspark                                              #Import library to Search for Spark Installation

findspark.init()                                              #Search Spark Installation

import pyspark                                                #Only run after findspark.init()

from pyspark.sql import SparkSession                          #Import of Spark Session
from pyspark import SparkContext as spark                     #Import the Regular Spark Contex
from pyspark.sql import SQLContext                            #Import the SQL Spark Contex
from pyspark.sql.functions import *                           #Import the SQL Spark Functions
from pyspark.sql.types import *                               #Import the SQL Spark Variables Types
spark = SparkSession.builder.getOrCreate()                   #Creating Spark Session

import os

from datetime import date                                     # Import date methoed
month_ref = date.today().month                                # Creating Current Month Variable
year_ref = date.today().year                                  # Creating Current Year Variable

sc = spark.sparkContext                                      #Initialize Spark Session / FRED

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Creating Variables

## Buckets

In [ ]:
SPG_INTEGRATION_INPUT_BUCKET = os.environ['INTEGRATION_INPUT_BUCKET']

In [ ]:
SPG_OUTPUT_BUCKET = os.environ['OUTPUT_BUCKET']

## Input Paths

In [ ]:
SPG_INTEGRATION_HANA_INPUT_PATH = "SPG_FACTS/SPG_HANA/SPG_HANA.parquet"

In [ ]:
SPG_INTEGRATION_BRANCH_INPUT_PATH = "SPG_GLOBAL/INPUT/tb_SPG_SUPPORT_CIDADE_FILIAL.csv"

In [ ]:
SPG_INTEGRATION_REGIONS_INPUT_PATH = "SPG_DIMENSIONS/SPG_REGIOES/SPG_RG_REGIOES/SPG_RG_REGIOES.parquet"

In [ ]:
SPG_INTEGRATION_PRODUCTS_INPUT_PATH = "SPG_DIMENSIONS/SPG_PRODUTOS/SPG_PD_PRODUTOS.parquet"

In [ ]:
SPG_INTEGRATION_GROSS_PROFIT_INPUT_PATH = "SPG_LB/LB_OrdemItem"

## Output Path

In [ ]:
SPG_OUTPUT_BUCKET_WATERFALL_OUTPUT_PATH = "SPG_WATERFALL/OUTPUT_ETL/watefall_parquet.parquet"

# DEFINED FUNCTIONS

In [102]:
def remove_some_hifen(col_name):
    removed_chars = ("-")
    regexp = "|".join('\{0}'.format(i) for i in removed_chars)
    return regexp_replace(col_name, regexp, "/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [103]:
def normalizing_orthography(col_name):
    removed_array = [('Á', 'A'), ('Ã', 'A'),('À', 'A'),('Â', 'A'),('Ä', 'A'),
                    ('É', 'E'),('È', 'E'),('Ê', 'E'),('Ë', 'E'),
                    ('Í', 'I'),('Ì', 'I'),('Î', 'I'),('Ï', 'I'),
                    ('Ó', 'O'),('Õ', 'O'), ('Ò', 'O'),('Ô', 'O'),('Ö', 'O'),
                    ('Ú', 'U'),('Ù', 'U'),('Û', 'U'),('Ü', 'U'),
                    ('Ç', 'C'),('`', ''),("'", ''),('-', ''),('/', ''),(' ', '')]
    r = col_name
    for a, b in removed_array:
        r = regexp_replace(r, a, b)
    return r

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# INPUTS

### Importing Tables

In [ ]:
df_hana_input = spark.read.parquet("s3a://"+SPG_INTEGRATION_INPUT_BUCKET+"/"+SPG_INTEGRATION_HANA_INPUT_PATH)

In [ ]:
df_regions_input = spark.read.parquet("s3a://"+SPG_INTEGRATION_INPUT_BUCKET+"/"+SPG_INTEGRATION_REGIONS_INPUT_PATH)

In [ ]:
df_products_input = spark.read.parquet("s3a://"+SPG_INTEGRATION_INPUT_BUCKET+"/"+SPG_INTEGRATION_PRODUCTS_INPUT_PATH)

In [ ]:
df_gross_profit_input = spark.read.parquet("s3a://"+SPG_INTEGRATION_INPUT_BUCKET+"/"+SPG_INTEGRATION_GROSS_PROFIT_INPUT_PATH)

# Processing Tables

## Hana

### apply temporary filters

In [105]:
# Filter the Sales Organizations within Smart Pricing scope
df_hana_input = df_hana_input.filter(df_hana_input.SalesOrg.like('%BRIN%') |
                       df_hana_input.SalesOrg.like('%BRIO%') |
                       df_hana_input.SalesOrg.like('%BRDI%') |
                       df_hana_input.SalesOrg.like('%BRDO%') |
                       df_hana_input.SalesOrg.like('%BRCC%') |
                       df_hana_input.SalesOrg.like('%BRCO%') |
                       df_hana_input.SalesOrg.like('%BRCG%') |
                       df_hana_input.SalesOrg.like('%BRGO%'))\
               .filter(df_hana_input.BillingType.like('%ZF2B%'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [106]:
# Import the last six months registers
df_hana_input = df_hana_input.filter(df_hana_input['BILDOCDATE']>=add_months(trunc(current_date(),"month"), -6))\
                       .filter(df_hana_input['BILDOCDATE']<add_months(trunc(current_date(),"month"), 1))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [107]:
#selecting specifics GPD's (business call)
df_hana_input = df_hana_input.filter(df_hana_input.GPDName.like('%BR-PERFIS ESTRUTURAIS%') |
                        df_hana_input.GPDName.like('%BR-CA50%') |
                        df_hana_input.GPDName.like('%BR-CA60%') |
                        df_hana_input.GPDName.like('%BR-RECOZIDO%') |
                        df_hana_input.GPDName.like('%BR-TEL TUBO%') |
                        df_hana_input.GPDName.like('%BR-TELA P/ CONCRETO%') |
                        df_hana_input.GPDName.like('%BR-CORTE E DOBR%') |
                        df_hana_input.GPDName.like('%BR-CONSTRUCAO CIVIL%') |
                        df_hana_input.GPDName.like('%BR-CA25%') |
                        df_hana_input.GPDName.like('%BR-PERFIS COMERCIAIS%') |
                        df_hana_input.GPDName.like('%BR-BTG%') |
                        df_hana_input.GPDName.like('%BR-MALHA POP%') |
                        df_hana_input.GPDName.like('%BR-TELA COLUNA%') |
                        df_hana_input.GPDName.like('%BR-TRELIÇA%') |
                        df_hana_input.GPDName.like('%BR-B.TREFILADA%') |
                        df_hana_input.GPDName.like('%BR-OVALADO%') |
                        df_hana_input.GPDName.like('%BR-GALVANIZADO%') |
                        df_hana_input.GPDName.like('%BR-BARRAS COMERCIAIS%') |
                        df_hana_input.GPDName.like('%BR-CAIXOTARIA%') |
                        df_hana_input.GPDName.like('%BR-ARTEFATOS%') |
                        df_hana_input.GPDName.like('%BR-FARPADO%') |
                        df_hana_input.GPDName.like('%BR-ELETRODO%') |
                        df_hana_input.GPDName.like('%BR-SOLDA - MIG%') |
                        df_hana_input.GPDName.like('%BR-CANTON A572%') |
                        df_hana_input.GPDName.like('%BR-GGS%') |
                        df_hana_input.GPDName.like('%BR-ARMADO%') |
                        df_hana_input.GPDName.like('%BR-CORDOALHA%') |
                        df_hana_input.GPDName.like('%BR-ESTACA PRANCHA%') |
                        df_hana_input.GPDName.like('%BR-ARAME PREGO%') |
                        df_hana_input.GPDName.like('%BR-CABEÇA DUPLA%') |
                        df_hana_input.GPDName.like('%BR-CORDOALHA AGRO%') |
                        df_hana_input.GPDName.like('%BR-GRAMPO%') |
                        df_hana_input.GPDName.like('%BR-COBREADOS%') |
                        df_hana_input.GPDName.like('%BR-CHAPA LQ%') |
                        df_hana_input.GPDName.like('%BR-UDC%') |
                        df_hana_input.GPDName.like('%BR-CHAPA ZN%') |
                        df_hana_input.GPDName.like('%BR-TELHA AZ%') |
                        df_hana_input.GPDName.like('%BR-TUBO ZN%') |
                        df_hana_input.GPDName.like('%BR-MARCENARIA%') |
                        df_hana_input.GPDName.like('%BR-PREGOES%') |
                        df_hana_input.GPDName.like('%BR-TELHEIRO%') |
                        df_hana_input.GPDName.like('%BR-COLUNA%') |
                        df_hana_input.GPDName.like('%BR-ESTRIBO%') |
                        df_hana_input.GPDName.like('%BR-ACESSORIOS%') |
                        df_hana_input.GPDName.like('%BR-CHAPA LCG%') |
                        df_hana_input.GPDName.like('%BR-CHAPA LF%') |
                        df_hana_input.GPDName.like('%BR-TUBO LF%') |
                        df_hana_input.GPDName.like('%BR-CHAPA LQ PISO%') |
                        df_hana_input.GPDName.like('%BR-BOBININHA%') |
                        df_hana_input.GPDName.like('%BR-ESPECIAIS%') |
                        df_hana_input.GPDName.like('%BR-BOBINA LQ%') |
                        df_hana_input.GPDName.like('%BR-FITA LQ%') |
                        df_hana_input.GPDName.like('%BR-BOBINA AZ%') |
                        df_hana_input.GPDName.like('%BR-AÇOS ESPECIAIS%') |
                        df_hana_input.GPDName.like('%BR-PARAFUSOS%') |
                        df_hana_input.GPDName.like('%BR-CIMENTO%') |
                        df_hana_input.GPDName.like('%BR-TUBO LQ%') |
                        df_hana_input.GPDName.like('%BR-TELHA ZN%') |
                        df_hana_input.GPDName.like('%BR-BTC GLV CP%') |
                        df_hana_input.GPDName.like('%BR-BOBINA ZN%') |
                        df_hana_input.GPDName.like('%BR-S-BARRA LAMINADA MÉDIA%') |
                        df_hana_input.GPDName.like('%BR-S-BARRA LAMINADA PESADA%') |
                        df_hana_input.GPDName.like('%BR-FITA LF%') |
                        df_hana_input.GPDName.like('%BR-FITA AZ%') |
                        df_hana_input.GPDName.like('%BR-S-BARRA LAMINADA LEVE%') |
                        df_hana_input.GPDName.like('%BR-ALAMBRADO%') |
                        df_hana_input.GPDName.like('%BR-SAPATA%') |
                        df_hana_input.GPDName.like('%BR-MOURÃO%') |
                        df_hana_input.GPDName.like('%BR-ATC CLARO IND%') |
                        df_hana_input.GPDName.like('%BR-POLIDO%') |
                        df_hana_input.GPDName.like('%BR-PERFIL BENEFICIADO%') |
                        df_hana_input.GPDName.like('%BR-BOBINA LF%') |
                        df_hana_input.GPDName.like('%BR-S-BARRA CTT%') |
                        df_hana_input.GPDName.like('%BR-CHAPA AZ%') |
                        df_hana_input.GPDName.like('%BR-FITA ZN%') |
                        df_hana_input.GPDName.like('%BR-LDA%') |
                        df_hana_input.GPDName.like('%BR-PIATINA CLARA%') |
                        df_hana_input.GPDName.like('%BR-S-BARRA DESCASCADA%') |
                        df_hana_input.GPDName.like('%BR-MESH%') |
                        df_hana_input.GPDName.like('%BR-S-BARRA FORJADA FINA%') |
                        df_hana_input.GPDName.like('%BR-S-BARRA RETIFICADA%') |
                        df_hana_input.GPDName.like('%BR-S-BARRA TREFILADA%') |
                        df_hana_input.GPDName.like('%BR-PLACA%') |
                        df_hana_input.GPDName.like('%BR-COIL%') |
                        df_hana_input.GPDName.like('%BR-S-FERRAMENTA%') |
                        df_hana_input.GPDName.like('%BR-S-FIO MÁQUINA STT%') |
                        df_hana_input.GPDName.like('%BR-ATC CLARO MOL COL%') |
                        df_hana_input.GPDName.like('%BR-ATC GLV CP%') |
                        df_hana_input.GPDName.like('%BR-B.CHATA LONGARINA%') |
                        df_hana_input.GPDName.like('%BR-BT FORJARIA BLOCOS%') |
                        df_hana_input.GPDName.like('%BR-TARUGO%') |
                        df_hana_input.GPDName.like('%BR-DRAWING%') |
                        df_hana_input.GPDName.like('%BR-ATC CLARO ENF ALG%') |
                        df_hana_input.GPDName.like('%BR-PIATINA GLV%') |
                        df_hana_input.GPDName.like('%BR-PERFIL DORMENTE%') |
                        df_hana_input.GPDName.like('%BR-ATC%') |
                        df_hana_input.GPDName.like('%BR-CHQ%') |
                        df_hana_input.GPDName.like('%BR-ALMA DE ELETRODO%') |
                        df_hana_input.GPDName.like('%BR-FIO MAQUINA%') |
                        df_hana_input.GPDName.like('%BR-CHQ BORO%') |
                        df_hana_input.GPDName.like('%BR-PERFIL GUIA ELEV%'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [108]:
#selecting specific GPM
df_hana_input = df_hana_input.filter(df_hana_input.GPM.like('%PERFIS_ESTRUTURAIS%') |
                                     df_hana_input.GPM.like('%VERGALHAO%') |
                                     df_hana_input.GPM.like('%AMPLIADOS%') |
                                     df_hana_input.GPM.like('%CORTE_DOBRA%') |
                                     df_hana_input.GPM.like('%PREGOS%') |
                                     df_hana_input.GPM.like('%B&P%') |
                                     df_hana_input.GPM.like('%ARAMES_AGRO%') |
                                     df_hana_input.GPM.like('%ARAMES_IND%') |
                                     df_hana_input.GPM.like('%PLANOS_LQ%') |
                                     df_hana_input.GPM.like('%PLANOS_REVESTIDOS%') |
                                     df_hana_input.GPM.like('%PLANOS_LF%') |
                                     df_hana_input.GPM.like('%PLANOS_LCG%') |
                                     df_hana_input.GPM.like('%FIO_MAQUINA%') |
                                     df_hana_input.GPM.like('%BT_FORJARIA%') |
                                     df_hana_input.GPM.like('%PLACAS%') |
                                     df_hana_input.GPM.like('%TARUGO%') | 
                                     df_hana_input.GPM.isNull())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [109]:
# Selecting Necessary Columns
df_hana_input=df_hana_input.select(df_hana_input.SoldToCustomer.alias("CLIENTE_COD")
                                  ,df_hana_input.SoldToCustomerName.alias("CLIENTE_DESC")
                                  ,df_hana_input.CorporateGroupName.alias("CLIENT_GROUP_DESC")
                                  ,df_hana_input.BILDOCDATE.alias("BILLING_DATE")
                                  ,df_hana_input.SalesDocNumber.alias("SALES_DOC_COD")
                                  ,df_hana_input.SalesDocItem.alias("SALES_DOC_ITEM_COD")
                                  ,df_hana_input.SalesOrg.alias("SALES_ORG_COD")
                                  ,df_hana_input.SoldToCity.alias("ISSUING_CITY")
                                  ,df_hana_input.SoldToState.alias("ISSUING_STATE")
                                  ,df_hana_input.GPD.alias("GPD_COD")
                                  ,df_hana_input.GPM.alias("GPM_DESC")
                                  ,df_hana_input.GPDName.alias("GPD_DESC")
                                  ,df_hana_input.Material.alias("MATERIAL_COD")
                                  ,df_hana_input.MaterialName.alias("MATERIAL_DESC")
                                  ,df_hana_input.MA_BillingQuantity_P.alias("QUANTITY_TON")
                                  ,df_hana_input.DA_CM_CD_ZD05_D.alias("ZD05")
                                  ,df_hana_input.IM_CM_CD_ICMS_D.alias("ICMS")
                                  ,df_hana_input.IM_CM_CD_PIS_D.alias("PIS")
                                  ,df_hana_input.AA_CM_CD_ZEAF_D.alias("ZEAF")
                                  ,df_hana_input.MA_CM_CD_BX41_D.alias("BX41")
                                  ,df_hana_input.MA_CM_RBV_D.alias("RBV")
                                  ,df_hana_input.MA_CM_RLV_D.alias("RLV")
                                  ,df_hana_input.MA_CM_PRECOBASE_ALCADA_P.alias("VERGE_INITIAL_PRICE")                       
                                  ,df_hana_input.DA_CM_CD_ZD01_D.alias("ZD01")
                                  ,df_hana_input.DA_CM_CD_ZD02_D.alias("ZD02")
                                  ,df_hana_input.DA_CM_CD_ZD03_D.alias("ZD03")
                                  ,df_hana_input.DA_CM_CD_ZD04_D.alias("ZD04")                       
                                  ,df_hana_input.DA_CM_CD_ZD06_D.alias("ZD06")
                                  ,df_hana_input.DM_CM_CD_ZD13_D.alias("ZD13")
                                  ,df_hana_input.MA_CM_PRECOLISTA_D.alias("LIST_PRICE")
                                  ,df_hana_input.AA_CM_CD_ZEB1_D.alias("ZEB1")
                                  ,df_hana_input.AA_CM_CD_ZEB2_D.alias("ZEB2")
                                  ,df_hana_input.AM_CM_CD_ZSU2_D.alias("ZSU2")
                                  ,df_hana_input.IM_CM_CD_COFINS_D.alias("COFINS")
                                  ,df_hana_input.MA_CM_CD_IPI_D.alias("IPI")
                                  ,df_hana_input.DM_CM_CD_ZD14_D.alias("ZD14")
                                  ,df_hana_input.IM_OUTROS_D.alias("OTHER_TAXES")
                                  ,df_hana_input.IM_CM_CD_BX95_D.alias("BX95"))\
.dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [110]:
# Create CLIENT_GROUP_DESC column
df_hana_input=df_hana_input.withColumn("CLIENT_GROUP_DESC", when(df_hana_input.CLIENT_GROUP_DESC.isNull()|
                                                                 df_hana_input.CLIENT_GROUP_DESC.like("GRUPO DE RENTABILIDADE GENÉRICO")
                                                                 ,col("CLIENTE_DESC")).otherwise(col("CLIENT_GROUP_DESC")))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [111]:
for col_name in ["ZD05"
                 ,"ZEAF"
                 ,"ZD04"
                 ,"ZD01"
                 ,"ZD02"
                 ,"ZD03"
                 ,"ZEB2"
                 ,"ZEB1"
                 ,"BX41"
                 ,"ICMS"
                 ,"COFINS"
                 ,"PIS"
                 ,"IPI"
                 ,"ZD06"
                 ,"ZD13"
                 ,"ZD14"
                 ,"RBV"
                 ,"ZSU2"
                 ,"LIST_PRICE"
                 ,"QUANTITY_TON"
                 ,"VERGE_INITIAL_PRICE"]:
    df_hana_input = df_hana_input.withColumn(col_name, col(col_name).cast('float'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [112]:
df_hana_input=df_hana_input.filter(df_hana_input.QUANTITY_TON > 0)\
.withColumn("SALES_ORG_COD", trim(df_hana_input.SALES_ORG_COD))\
.distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [113]:
df_hana_input.write.partitionBy("SALES_ORG_COD")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [114]:
df_hana = df_hana_input.repartition("SALES_ORG_COD")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### SPG Regions

In [116]:
df_regions_input=df_regions_input.withColumn("UPPER_NAME", normalizing_orthography(upper(df_regions_input["IBGE_SAP_CITY_NAME"])))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [117]:
df_regions_input=df_regions_input.select(df_regions_input.UPPER_NAME
                                         ,df_regions_input.BRANCH
                                         ,df_regions_input.IBGE_UF_ACRONYMS)\
.dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [118]:
df_regions_input.write.partitionBy("IBGE_UF_ACRONYMS")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [119]:
df_regions = df_regions_input.repartition("IBGE_UF_ACRONYMS")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### SPG Products

In [120]:
df_products_input = spark.read.parquet("s3a://smartpricinggerdaubucket/SPG_DIMENSIONS/SPG_PRODUTOS/SPG_PD_PRODUTOS.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [121]:
# Remove duplicates
df_products_input=df_products_input.select(trim(df_products_input.DESC_GPD).alias("DESC_GPD")
                                           ,df_products_input.COD_GPD
                                           ,trim(df_products_input.DESC_GPM).alias("GPM_DESC_prod_drop")
                                           ,df_products_input.COD_MATERIAL.alias("COD_MATERIAL_drop")
                                           ,trim(df_products_input.SALES_ORG_COD).alias("SALES_ORG_COD_drop"))\
.dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [122]:
df_products_input.write.partitionBy("COD_GPD")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [123]:
df_products = df_products_input.repartition("COD_GPD")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### SPG Gross Profit

In [125]:
# Create column MONTH_YEAR_LB
df_gross_profit_input = df_gross_profit_input.withColumn("MONTH_YEAR_LB", concat(when(col("month") >= "10", col("month"))
                                                                                 .otherwise(concat(lit("0"), col("month")))
                                                                                 ,lit("-")
                                                                                 ,col("year")))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [126]:
# Import and rename gross profit columns
df_gross_profit_input=df_gross_profit_input.select(df_gross_profit_input.sales_number.alias("SALES_NUMBER")
                                                   ,df_gross_profit_input.sales_item.alias("SALES_ITEM")
                                                   ,df_gross_profit_input.MONTH_YEAR_LB.alias("MONTH_YEAR_LB")
                                                   ,df_gross_profit_input.COGS_TOTAL.alias("OP_COST").cast('float')
                                                   ,df_gross_profit_input.FREIGHT_Full.alias("FREIGHT").cast('float')
                                                   ,df_gross_profit_input.GROSS_PROFIT_BW.alias("GROSS_PROFIT_LB").cast('float'))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [127]:
df_gross_profit_input.write.partitionBy("MONTH_YEAR_LB")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [128]:
df_gross_profit = df_gross_profit_input.repartition("MONTH_YEAR_LB")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [129]:
# Create KEY_LB_DROP column
df_gross_profit = df_gross_profit.withColumn("key_Lb_drop", concat(col("MONTH_YEAR_LB")
                                                                   ,lit("_")
                                                                   ,col("SALES_NUMBER").cast("int")
                                                                   ,lit("_")
                                                                   ,col("SALES_ITEM").cast("int")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# TRANSFORM HANA

In [130]:
# Correcting "Ouro Branco" Sales Org 
    # BRIO -> BRIN
    # BRCO -> BRCC
    # BRDO -> BRDI
    # BRGO -> BRCG
df_hana = df_hana.withColumn("SALES_ORG_COD", when(df_hana.SALES_ORG_COD.like("BRCO"), "BRCC")\
                                              .otherwise(when(df_hana.SALES_ORG_COD.like("BRGO"), "BRCG")\
                                                        .otherwise(when(df_hana.SALES_ORG_COD.like("BRIO"), "BRIN")\
                                                                  .otherwise(when(df_hana.SALES_ORG_COD.like("BRDO"), "BRDI")
                                                                            .otherwise(df_hana.SALES_ORG_COD)))))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [131]:
# Getting GPM_DESC Field
df_hana=df_hana.join(df_products, (trim(df_products.SALES_ORG_COD_drop)==trim(df_hana.SALES_ORG_COD))&
                                  (df_products.COD_MATERIAL_drop.cast("int")==df_hana.MATERIAL_COD.cast("int")), how="left")\
               .drop("COD_MATERIAL_drop")\
               .drop("SALES_ORG_COD_drop")\
               .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [132]:
# Correcting GPM_DESC Column
df_hana=df_hana.withColumn("GPM_DESC", when(df_hana.GPM_DESC_prod_drop.isNull(), df_hana.GPM_DESC)\
                                           .otherwise(df_hana.GPM_DESC_prod_drop))\
               .withColumn("GPD_COD", when(df_hana.COD_GPD.isNull(), df_hana.GPD_COD)\
                                           .otherwise(df_hana.COD_GPD))\
               .withColumn("GPD_DESC", when(df_hana.DESC_GPD.isNull(), df_hana.GPD_DESC)\
                                           .otherwise(df_hana.DESC_GPD))\
               .drop("GPM_DESC_prod_drop")\
               .drop("DESC_GPD")\
               .drop("COD_GPD")\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# TRANSFORM WATERFALL

In [133]:
df_hana=df_hana.filter(df_hana.QUANTITY_TON > 0)\
                     .withColumn("SALES_ORG_COD", trim(df_hana.SALES_ORG_COD))\
                     .distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [134]:
# Unite hana and regions table
df_hana = df_hana.join(df_regions, (normalizing_orthography(upper(df_hana["ISSUING_CITY"])) == df_regions["UPPER_NAME"])&
                                   (trim(df_hana["ISSUING_STATE"]) == trim(df_regions["IBGE_UF_ACRONYMS"])), how='left')\
                     .drop("UPPER_NAME")\
                     .drop("IBGE_UF_ACRONYMS")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [135]:
df_hana.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[CLIENTE_COD: string, CLIENTE_DESC: string, CLIENT_GROUP_DESC: string, BILLING_DATE: string, SALES_DOC_COD: string, SALES_DOC_ITEM_COD: string, SALES_ORG_COD: string, ISSUING_CITY: string, ISSUING_STATE: string, GPD_COD: string, GPM_DESC: string, GPD_DESC: string, MATERIAL_COD: string, MATERIAL_DESC: string, QUANTITY_TON: float, ZD05: float, ICMS: float, PIS: float, ZEAF: float, BX41: float, RBV: float, RLV: double, PRECO_BASE_ALCADA: float, ZD01: float, ZD02: float, ZD03: float, ZD04: float, ZD06: float, ZD13: float, LIST_PRICE: float, ZEB1: float, ZEB2: float, ZSU2: float, COFINS: float, IPI: float, ZD14: float, OTHER_TAXES: double, BX95: double, BRANCH: string]

In [136]:
# Create BRANCH column
df_hana = df_hana.withColumn("BRANCH", when(df_hana.BRANCH.isNull(), lit("Filial Não Encontrada"))\
                                                   .otherwise(df_hana.BRANCH))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [137]:
# Create MONTH_YEAR column
df_hana = df_hana.withColumn("MONTH_YEAR", concat(when(month(df_hana.BILLING_DATE) >= "10", month(df_hana.BILLING_DATE))
                                                            .otherwise(concat(lit("0"), month(df_hana.BILLING_DATE)))
                                                ,lit("-")
                                                ,year(df_hana.BILLING_DATE) ))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [138]:
df_hana=df_hana.drop("SALES_ORG_DESC"
                     ,"VERGE_INITIAL_PRICE"
                     ,"SALES_DISCOUNT_TAXES"
                     ,"GDD_DESC"
                     ,"GPP_DESC"
                     ,"ZP30"
                     ,"ZP20"
                     ,"CUSTOMER_GROUP_DESC")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Group BY ordem_item por data

In [139]:
df_hana=df_hana.groupBy(df_hana.SALES_ORG_COD
                        ,df_hana.SALES_DOC_COD
                        ,df_hana.SALES_DOC_ITEM_COD
                        ,df_hana.ISSUING_STATE
                        ,df_hana.ISSUING_CITY
                        ,df_hana.CLIENTE_COD
                        ,df_hana.CLIENTE_DESC
                        ,df_hana.CLIENT_GROUP_DESC
                        ,df_hana.GPD_COD
                        ,df_hana.GPD_DESC
                        ,df_hana.MATERIAL_COD
                        ,df_hana.MATERIAL_DESC
                        ,df_hana.GPM_DESC
                        ,df_hana.BRANCH
                        ,df_hana.BILLING_DATE
                        ,df_hana.MONTH_YEAR)\
.agg(avg(col("LIST_PRICE")).alias("LIST_PRICE")
     ,sum(col("IPI")).alias("IPI")
                         ,sum(col("PIS")).alias("PIS")
                         ,sum(col("COFINS")).alias("COFINS")
                         ,sum(col("ICMS")).alias("ICMS")
                         ,sum(col("ZD02")).alias("ZD02")
                         ,sum(col("ZD03")).alias("ZD03")
                         ,sum(col("ZD01")).alias("ZD01")
                         ,sum(col("ZD04")).alias("ZD04")
                         ,sum(col("ZD05")).alias("ZD05")
                         ,sum(col("ZEAF")).alias("ZEAF")
                         ,sum(col("ZEB1")).alias("ZEB1")
                         ,sum(col("ZEB2")).alias("ZEB2")
                         ,sum(col("ZD13")).alias("ZD13")
                         ,sum(col("ZD14")).alias("ZD14")
                         ,sum(col("ZD06")).alias("ZD06")
                         ,sum(col("ZSU2")).alias("ZSU2")
                         ,sum(col("RBV")).alias("RBV")
                         ,sum(col("BX41")).alias("BX41")
                         ,sum(col("RLV")).alias("RLV")
                         ,sum(col("QUANTITY_TON")).alias("QUANTITY_TON")
                         ,sum(col("OTHER_TAXES")).alias("OTHER_TAXES")
                         ,sum(col("BX95")).alias("BX95"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [140]:
# Create KEY_DROP column 
df_hana = df_hana.withColumn("key_drop", concat(col("MONTH_YEAR")
                                                ,lit("_")
                                                ,col("SALES_DOC_COD").cast("int")
                                                ,lit("_")
                                                ,col("SALES_DOC_ITEM_COD").cast("int")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [141]:
# Unite gross profit and hana table
df_hana = df_hana.join(df_gross_profit.select(df_gross_profit.key_Lb_drop
                                              ,df_gross_profit.GROSS_PROFIT_LB
                                              ,df_gross_profit.FREIGHT
                                              ,df_gross_profit.OP_COST)
                             ,(df_hana["key_drop"] == df_gross_profit["key_Lb_drop"]), how='left')\
                .drop("key_Lb_drop")\
                .drop("key_drop")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [142]:
df_hana.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[SALES_ORG_COD: string, SALES_DOC_COD: string, SALES_DOC_ITEM_COD: string, ISSUING_STATE: string, ISSUING_CITY: string, CLIENTE_COD: string, CLIENTE_DESC: string, CLIENT_GROUP_DESC: string, GPD_COD: string, GPD_DESC: string, MATERIAL_COD: string, MATERIAL_DESC: string, GPM_DESC: string, BRANCH: string, BILLING_DATE: string, MONTH_YEAR: string, LIST_PRICE: double, IPI: double, PIS: double, COFINS: double, ICMS: double, ZD02: double, ZD03: double, ZD01: double, ZD04: double, ZD05: double, ZEAF: double, ZEB1: double, ZEB2: double, ZD13: double, ZD14: double, ZD06: double, ZSU2: double, RBV: double, BX41: double, RLV: double, QUANTITY_TON: double, OTHER_TAXES: double, BX95: double, GROSS_PROFIT_LB: float, FREIGHT: float, OP_COST: float]

In [143]:
# Replace null with 0
df_hana = df_hana.fillna({'GROSS_PROFIT_LB':'0'
                         ,'FREIGHT':'0'
                         ,'OP_COST':'0'})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [144]:
df_hana = df_hana.withColumn("GROSS_PROFIT_LB", df_hana["GROSS_PROFIT_LB"].cast("float"))\
                 .withColumn("FREIGHT", df_hana["FREIGHT"].cast("float"))\
                 .withColumn("RLV", df_hana["RLV"].cast("float"))\
                 .withColumn("OP_COST", df_hana["OP_COST"].cast("float"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [145]:
#Creating TAXES
df_hana=df_hana.withColumn("CALCULATED_TAXES",(col('ICMS')
                                              + col('COFINS')
                                              + col("PIS")
                                              + col("OTHER_TAXES")
                                              + col("BX95")).cast('float'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [146]:
# Create SEGMENT_PRICE column
df_hana=df_hana.withColumn("SEGMENT_PRICE",(col('LIST_PRICE')+
                                              col('ZD02')+ 
                                              col('ZD03')).cast('float'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [147]:
# Create FULL_CUSTOMER_PRICE column
df_hana=df_hana.withColumn("FULL_CUSTOMER_PRICE",(col('SEGMENT_PRICE')+
                                                  col('ZD01')+
                                                  col('ZD04')+
                                                  col('ZD05')+
                                                  col('ZEAF')+
                                                  col('ZEB1')+
                                                  col('ZEB2')).cast('float'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [148]:
# Create RBV_WATERFALL column
df_hana=df_hana.withColumn("RBV_WATERFALL",(col('FULL_CUSTOMER_PRICE')+
                                          col('ZD13')+
                                          col('ZD14')+
                                          col('ZD06')+
                                          col('ZSU2')).cast('float'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [149]:
# Create RLV_WATERFALL column
df_hana=df_hana.withColumn("RLV_WATERFALL",(when(df_hana.SALES_ORG_COD.like("BRCG"), col("RBV_WATERFALL")
                                                                          -abs(col("CALCULATED_TAXES")))\
                                        .otherwise(col("RBV_WATERFALL")
                                                    -abs(col("CALCULATED_TAXES")))).cast('float'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [150]:
# Create GROSS_PROFIT_WATERFALL column
df_hana=df_hana.withColumn("GROSS_PROFIT_WATERFALL",(col('RLV_WATERFALL')
                                                    - abs(col('OP_COST'))
                                                    - abs(col('FREIGHT'))).cast('float'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### remover

In [151]:
df_hana = df_hana.withColumn("GROSS_PROFIT_ORIGINAL", col('GROSS_PROFIT_LB'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [152]:
# Dividing by quantity and by ton
for col_name in ["GROSS_PROFIT_WATERFALL"
                 ,"GROSS_PROFIT_LB"
                 ,"LIST_PRICE"
                 ,"ZD02"
                 ,"ZD03"
                 ,"SEGMENT_PRICE"
                 ,"ZD01"
                 ,"ZD04"
                 ,"ZD05"
                 ,"ZEAF"
                 ,"ZEB1"
                 ,"IPI"
                 ,"PIS"
                 ,"COFINS"
                 ,"ICMS"
                 ,"ZEB2"
                 ,"FULL_CUSTOMER_PRICE"
                 ,"ZD13"
                 ,"ZD14"
                 ,"ZD06"
                 ,"ZSU2"
                 ,"RBV"
                 ,"RBV_WATERFALL"
                 ,"CALCULATED_TAXES"
                 ,"BX41"
                 ,"RLV"
                 ,"RLV_WATERFALL"
                 ,"FREIGHT"
                 ,"OP_COST"
                 ,"OTHER_TAXES"
                 ,"BX95"]:
    df_hana = df_hana.withColumn(col_name, (col(col_name)/col("QUANTITY_TON")).cast('float'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [153]:
df_hana = df_hana.withColumn("COUNTRY", lit("Brazil"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [154]:
df_waterfall=df_hana.withColumn("GROSS_PROFIT_WATERFALL", (col("GROSS_PROFIT_WATERFALL")).cast('float'))\
                            .withColumn("GROSS_PROFIT_LB", (col("GROSS_PROFIT_LB")).cast('float'))\
                            .withColumn("GROSS_PROFIT_ORIGINAL", (col("GROSS_PROFIT_ORIGINAL")).cast('float'))\
                            .withColumn("IPI", abs(col("IPI")).cast('float'))\
                            .withColumn("PIS", abs(col("PIS")).cast('float'))\
                            .withColumn("COFINS", abs(col("COFINS")).cast('float'))\
                            .withColumn("ICMS", abs(col("ICMS")).cast('float'))\
                            .withColumn("LIST_PRICE", abs(col("LIST_PRICE")).cast('float'))\
                            .withColumn("ZD02", abs(col("ZD02")).cast('float'))\
                            .withColumn("ZD03", abs(col("ZD03")).cast('float'))\
                            .withColumn("SEGMENT_PRICE", abs(col("SEGMENT_PRICE")).cast('float'))\
                            .withColumn("ZD01", abs(col("ZD01")).cast('float'))\
                            .withColumn("ZD04", abs(col("ZD04")).cast('float'))\
                            .withColumn("ZD05", abs(col("ZD05")).cast('float'))\
                            .withColumn("ZEAF", abs(col("ZEAF")).cast('float'))\
                            .withColumn("ZEB1", abs(col("ZEB1")).cast('float'))\
                            .withColumn("ZEB2", abs(col("ZEB2")).cast('float'))\
                            .withColumn("FULL_CUSTOMER_PRICE", abs(col("FULL_CUSTOMER_PRICE")).cast('float'))\
                            .withColumn("ZD13", abs(col("ZD13")).cast('float'))\
                            .withColumn("ZD14", abs(col("ZD14")).cast('float'))\
                            .withColumn("ZD06", abs(col("ZD06")).cast('float'))\
                            .withColumn("ZSU2", abs(col("ZSU2")).cast('float'))\
                            .withColumn("RBV", abs(col("RBV")).cast('float'))\
                            .withColumn("RBV_WATERFALL", abs(col("RBV_WATERFALL")).cast('float'))\
                            .withColumn("BX41", abs(col("BX41")).cast('float'))\
                            .withColumn("RLV", abs(col("RLV")).cast('float'))\
                            .withColumn("RLV_WATERFALL", abs(col("RLV_WATERFALL")).cast('float'))\
                            .withColumn("OP_COST", abs(col("OP_COST")).cast('float'))\
                            .withColumn("FREIGHT", abs(col("FREIGHT")).cast('float'))\
                            .withColumn("QUANTITY_TON", abs(col("QUANTITY_TON")).cast('float'))\
                            .withColumn("CLIENTE_COD", abs(col("CLIENTE_COD")).cast(IntegerType()))\
                            .withColumn("GPD_COD", abs(col("GPD_COD")).cast(IntegerType()))\
                            .withColumn("MATERIAL_COD", abs(col("MATERIAL_COD")).cast(IntegerType()))\
                            .withColumn("BX95", abs(col("BX95")).cast('float'))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [155]:
df_waterfall.write.parquet("s3a://"+SPG_OUTPUT_BUCKET+"/"+SPG_OUTPUT_BUCKET_WATERFALL_OUTPUT_PATH, mode = "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…